# Week Three Assignment - Question 2

## Steps followed in creating the required dataframe

* Install all the libraries
* Download all the modules
* Read data from the Wikipedia Page
* Creating BeautifulSoup object 
* Reading and coverting BeautifulSoup oject data into dataframe
* Cleaning the data frame to remove rows with no Borough name
* Concatenating Neighborhood names which have the same Postal Code
* Adding Neighborhood names for rows missing the same
* Verify final output is corrrect and also print the shape of the final dataframe
* Reading the csv file with latitude and longitude value
* Modify column names inorder to perform the merge
* Merging data to create the new data set with latitude and longitude data

In [ ]:
# intalling beaitifulsoup library
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge requests --yes
#!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge geopy --yes 

In [74]:
# importing necessary modules
import requests as req
import pandas as pd
import numpy as np
import lxml
from bs4 import BeautifulSoup

In [75]:
# url for the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = req.get(url).text

In [76]:
# cretaing a beautifulsoup obect
soup = BeautifulSoup(source,'lxml')

In [77]:
# creating & loading data into dataframe
pcode_df = pd.DataFrame()
i = 0
j = 0
table_pcode = soup.find('table')
for table_data in table_pcode.find_all('td'):
    col_data = table_data.text
    pcode_df.loc[i,j] = col_data
    j = j + 1
    if (j==3):
        i = i + 1
        j = 0

In [78]:
# adding column names to the data frame
pcode_df.columns = ('PostalCode','Borough','Neighborhood')

In [79]:
# removing the \n haracter from the end of the string
pcode_df['Neighborhood'] = pcode_df['Neighborhood'].str.slice(stop = -1)

In [80]:
# removing the rows with borough value as not assigned
pcode_df=pcode_df[pcode_df.Borough != 'Not assigned']
pcode_df = pcode_df.reset_index(drop=True)

In [81]:
# concatenating neighborhoods with same Postal Code
new_df = pd.DataFrame()
new_df = pcode_df.groupby('PostalCode').agg({'Neighborhood':lambda x: tuple(x)})
new_df.reset_index(inplace=True)

In [82]:
# converting the tuple into a string data type
new_df['Neighborhood'] = new_df['Neighborhood'].astype(str)

In [83]:
#removing the end and begenning braces and the single quotes
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(stop = -2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(start = 2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.replace("'","")

In [ ]:
# creating a data frame of unique postal code and borough combination
df_borough = pcode_df
df_borough = df_borough.drop('Neighborhood',axis=1)
df_borough.drop_duplicates(inplace=True)
df_borough.reset_index(drop=True)

In [85]:
# creation of the final data frame by combining two intermediate data frames using Postal code as key
final_pcode_df = pd.DataFrame()
final_pcode_df = df_borough.merge(new_df, on='PostalCode')

In [86]:
# updating the data for rows missing Neighborhood values
loc_Neigh_NA = final_pcode_df.Neighborhood == 'Not assigned'
final_pcode_df.loc[loc_Neigh_NA,'Neighborhood'] = final_pcode_df.loc[loc_Neigh_NA,'Borough']
final_pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [87]:
# shape of the final dataframe
print('Shape of the Final DataFrame',final_pcode_df.shape)

Shape of the Final DataFrame (103, 3)


Attempt with geocode was unsuccesful. Attempt with geopy was also unsuccesful. Finally used the csv file provided to complete the assignmentto get the latitude and logitude information

In [91]:
# reading the csv file with latitude and longitude value
lat_long_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [92]:
# modify column names inorder to perform the merge
lat_long_df.columns = ('PostalCode','Latitude','Longitude')

In [93]:
# merging data to create the new data set with latitude and longitude data
final_lat_long_df = final_pcode_df.merge(lat_long_df , on='PostalCode')
final_lat_long_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
